#### 사용자 정의 데이터셋
- Pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하(H/W, S/W) 및 많은 시간 소요에 대한 해결책으로 제시  
- 대량 데이터셋 전용 처리 모듈 제공
- Dataset과 DataLoader
    * Dataset => 데이터셋 전처리, 텐서화 등의 작업 진행
    * DataLoader => Dataset 인스턴스를 사용해서 배치크기 만큼 데이터를 추출함

[1] 모듈 로딩 및 데이터 준비

In [67]:
# 모듈 로딩
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [68]:
# 데이터
FILE_PATH = 'iris.csv'


In [69]:
# csv => DataFrmae
irisDF = pd.read_csv(FILE_PATH)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [70]:
# 타겟 컬럼 수치화 == > LabelEncoder
from sklearn.preprocessing import LabelEncoder      # 타겟 컬럼 수치화

In [71]:
lb = LabelEncoder()
lb.fit(irisDF['variety'])
irisDF['variety']=lb.transform(irisDF['variety'])

[2] 사용자 정의 데이터셋 클래스 생성

In [72]:
# 클래스 목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스 이름 : customDataSet
# 부모클래스 : torch.utils.data.Dataset
# 매개 변수: featureDF, targetDF

In [73]:
class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_feaures=featureDF.shape[1]

    # 데이터의 개수 반환 메서드
    def __len__(self) :
        return self.n_rows

    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환!!!
    def __getitem__(self, idx) :
        featureTS=torch.FloatTensor( self.featureDF.iloc[idx].values)
        targetTS=torch.FloatTensor( self.targetDF.iloc[idx].values)
        return featureTS, targetTS

In [80]:
# 함수기능 : 파일 확장자별 데이터프레임 변환 가능
# 함수이름 : convertDataFrame
# 매개 변수 : data_path
# 함수결과 : DataFrame
def convertDataFrame(data_path, exit_header=0):
    ext = data_path.rsplit('.')[-1]
    if ext == 'csv':
        return pd.read_csv(data_path, header= exit_header)
    elif ext == 'json':
        return pd.read_json(data_path, header= exit_header)
    elif ext in ['xlsx', 'xls']:
        return pd.read_excel(data_path, header= exit_header)
    else:
        return pd.read_table(data_path, header= exit_header)

In [ ]:
# 클래스기능 : 파일기반 데이터셋
# 클래스이름 : FileDataset
# 매개 변수 : data_path 파일경로
# 부모클래스 : utils.data.Dataset
# ------------------------------------
class FileDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, data_path):
        super().__init__()
        dataDF=convertDataFrame(data_path)
        self.featuresDF=dataDF[dataDF.columns[:-1]]
        self.targetDF=dataDF[dataDF.columns[-1:]]

        self.n_features=self.featuresDF.shape[1]
        self.n_rows=self.featuresDF.shape[0]

    # 데이터의 개수 반환 메서드
    def __len__(self) :
        return self.n_rows

    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환!!!
    def __getitem__(self, idx) :
        featureTS=torch.FloatTensor( self.featuresDF.iloc[idx].values)
        targetTS=torch.FloatTensor( self.targetDF.iloc[idx].values)
        return featureTS, targetTS

[3] 데이터셋 인스턴스 생성

In [74]:
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[[irisDF.columns[-1]]]

print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

featureDF => (150, 4), targetDF => (150, 1)


In [75]:
irisDS = CustomDataset(featureDF, targetDF)

In [76]:
irisDS.n_feaures, irisDS.n_rows

(4, 150)

In [77]:
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [78]:
## 필요한 것 : Dataset 인스턴스, Batch_size=1 기본값
irisDL=DataLoader(irisDS, batch_size=3)

In [79]:
for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)
    break

torch.Size([3, 4]) torch.Size([3, 1])
